In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



**Data Augmentation**
---
**About the data:**
---
The dataset contains 2 folders: yes and no which contains 253 Brain MRI Images. The folder yes contains 155 Brain MRI Images that are tumorous and the folder no contains 98 Brain MRI Images that are non-tumorous. 

Since this is a small dataset, I used data augmentation in order to create more images.

Also, we could solve the data imbalance issue (since 61% of the data belongs to the tumorous class) using data augmentation.

**Import Necessary Modules**

In [17]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import cv2
import imutils
import matplotlib.pyplot as plt
from os import listdir
import time    
import os
%matplotlib inline

Chnage the Directory

In [29]:
!pwd
import os
os.chdir('/content/drive/My Drive/')
!pwd

/content/drive/My Drive
/content/drive/My Drive


Make new directory to store generated images 

In [ ]:
os.mkdir('augmented data')


In [34]:
os.chdir('/content/drive/My Drive/augmented data')
os.mkdir('yes')
os.mkdir('no')

In [35]:
def augment_data(file_dir, n_generated_samples, save_to_dir):
    
    #from keras.preprocessing.image import ImageDataGenerator
    #from os import listdir
    
    data_gen = ImageDataGenerator(rotation_range=10, 
                                  width_shift_range=0.1, 
                                  height_shift_range=0.1, 
                                  shear_range=0.1, 
                                  brightness_range=(0.3, 1.0),
                                  horizontal_flip=True, 
                                  vertical_flip=True, 
                                  fill_mode='nearest'
                                 )

    
    for filename in listdir(file_dir):
        # load the image
        image = cv2.imread(file_dir + '/' + filename)
        # reshape the image
        image = image.reshape((1,)+image.shape)
        # prefix of the names for the generated sampels.
        save_prefix = 'aug_' + filename[:-4]
        # generate 'n_generated_samples' sample images
        i=0
        for batch in data_gen.flow(x=image, batch_size=1, save_to_dir=save_to_dir, 
                                           save_prefix=save_prefix, save_format='jpg'):
            i += 1
            if i > n_generated_samples:
                break


Remember that 61% of the data (155 images) are tumorous. And, 39% of the data (98 images) are non-tumorous.
So, in order to balance the data we can generate 9 new images for every image that belongs to 'no' class and 6 images for every image that belongs the 'yes' class.

In [36]:

augmented_data_path = '/content/drive/My Drive/augmented data/'
# augment data for the examples with label equal to 'yes' representing tumurous examples
augment_data(file_dir='/content/drive/My Drive/brain_tumor_dataset/yes', n_generated_samples=6, save_to_dir='/content/drive/My Drive/augmented data/yes/')
# augment data for the examples with label equal to 'no' representing non-tumurous examples
augment_data(file_dir='/content/drive/My Drive/brain_tumor_dataset/no', n_generated_samples=9, save_to_dir='/content/drive/My Drive/augmented data/no/')

Let's see how many tumorous and non-tumorous examples after performing data augmentation:

In [37]:
def data_summary(main_path):
    
    yes_path = main_path+'/yes'
    no_path = main_path+'/no'
        
    # number of files (images) that are in the the folder named 'yes' that represent tumorous (positive) examples
    m_pos = len(listdir(yes_path))
    # number of files (images) that are in the the folder named 'no' that represent non-tumorous (negative) examples
    m_neg = len(listdir(no_path))
    # number of all examples
    m = (m_pos+m_neg)
    
    pos_prec = (m_pos* 100.0)/ m
    neg_prec = (m_neg* 100.0)/ m
    
    print(f"Number of examples: {m}")
    print(f"Percentage of positive examples: {pos_prec}%, number of pos examples: {m_pos}") 
    print(f"Percentage of negative examples: {neg_prec}%, number of neg examples: {m_neg}")

In [38]:
data_summary(augmented_data_path)

Number of examples: 2064
Percentage of positive examples: 52.51937984496124%, number of pos examples: 1084
Percentage of negative examples: 47.48062015503876%, number of neg examples: 980
